In [1]:
import os
import pandas as pd
import pyspark
import pyspark.ml as ML
conf = pyspark.SparkConf().setMaster("local[*]").setAppName("HMP")
sc = pyspark.SparkContext(conf = conf)
spark=pyspark.sql.SparkSession(sc)

In [2]:
OriginalDataSet=spark.read.parquet("DataSet.parquet")
OriginalDataSet.show()

+---+---+---+--------+-----+--------------+------------------------+-----------------------+
|  X|  Y|  Z|   Class|Label| One_Hot_Lanel|Input_Vector_Unprocessed|Input_Vector_Normalized|
+---+---+---+--------+-----+--------------+------------------------+-----------------------+
| 33| 36| 51|Eat_meat|  5.0|(13,[5],[1.0])|        [33.0,36.0,51.0]|      [0.275,0.3,0.425]|
| 33| 36| 51|Eat_meat|  5.0|(13,[5],[1.0])|        [33.0,36.0,51.0]|      [0.275,0.3,0.425]|
| 33| 35| 53|Eat_meat|  5.0|(13,[5],[1.0])|        [33.0,35.0,53.0]|   [0.27272727272727...|
| 31| 37| 52|Eat_meat|  5.0|(13,[5],[1.0])|        [31.0,37.0,52.0]|   [0.25833333333333...|
| 32| 36| 52|Eat_meat|  5.0|(13,[5],[1.0])|        [32.0,36.0,52.0]|   [0.26666666666666...|
| 32| 36| 51|Eat_meat|  5.0|(13,[5],[1.0])|        [32.0,36.0,51.0]|   [0.26890756302521...|
| 32| 36| 51|Eat_meat|  5.0|(13,[5],[1.0])|        [32.0,36.0,51.0]|   [0.26890756302521...|
| 33| 36| 53|Eat_meat|  5.0|(13,[5],[1.0])|        [33.0,36.0,53.0]|  

As seen below, the relationship between X,Y,Z is not linear.

Nonetheless, we will proceed to build a linear model that takes in X,Y and predicts Z, but givent the non-linear nature of the relations, we do not expect great performance from this model.

In [3]:
temp_pd=OriginalDataSet.sample(False,0.01).select("X Y Z".split()).toPandas()
from pandas.plotting import scatter_matrix
stuff = scatter_matrix(temp_pd, alpha=0.7, figsize=(6, 6), diagonal='hist')

In [4]:
Temp=ML.feature.VectorAssembler(inputCols=["X","Y"],outputCol="features").transform(OriginalDataSet)
Temp.createOrReplaceTempView("HMP")
LR_Data=spark.sql("""
SELECT features,Z as label
from HMP
""")

In [5]:
LR_Model=ML.regression.LinearRegression().fit(LR_Data)

In [6]:
Evaluation=LR_Model.evaluate(LR_Data)
print("The value of R2 is {}".format(round(Evaluation.r2,3)))

The value of R2 is 0.385


The poor performance is not surprising, as previously discussed.